# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

In [213]:
# Instala a biblioteca "TA-Lib" que será responsável por fornecer a implementação de alguns indicadores técnicos.
!pip install Ta-Lib

Defaulting to user installation because normal site-packages is not writeable


In [214]:
# Instala a biblioteca "tensorflow" que será responsável por fornecer a implementação dos métodos necessários para a construção da LSTM.
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


## *Importação de bibliotecas*

In [402]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Importa a biblioteca que será utilizada para a obtenção dos dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa o módulo da biblioteca "datetime" que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa o módulo da biblioteca "typing" que será utilizado para tipar parâmetros opcionais de funções.
from typing import Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np
# Importa a biblioteca que será utilizada para calcular algumas métricas de análise técnica.
import talib
# Importa o módulo da biblioteca "sklearn" que será utilizado para escalar as features dos DataFrames que serão usados no modelo LSTM.
from sklearn.preprocessing import MinMaxScaler
# Importa um dos módulos da biblioteca "tensorflow" que será utilizado para criar o modelo LSTM. 
from keras.models import Sequential
# Importa um dos módulos da biblioteca "tensorflow" que será utilizado para construir as camadas que farão parte da arquitetura do modelo LSTM 
# a ser usado. 
from keras.layers import LSTM, Dense, Dropout, Input
# Importa a biblioteca que será utilizada para criar gráficos interativos.
import plotly.graph_objects as go
# Importa a função "mean_squared_error" da biblioteca sklearn.metrics.
from sklearn.metrics import mean_squared_error

# **Definindo a classe Ticker**

In [361]:
class Ticker:
    '''
        Description:
            A classe "Ticker" representa um ativo financeiro identificado por um símbolo único (ticker). 
            Ela é responsável por baixar os dados históricos de preços e calcular vários indicadores técnicos, 
            como retornos aritméticos e logarítmicos, Bandas de Bollinger, RSI (Índice de Força Relativa), 
            ATR (Average True Range) e Momentum. O objetivo principal da classe é fornecer um conjunto padronizado de dados 
            e indicadores para análise de ativos financeiros. Além disso, valida se os dados extraídos estão de acordo com 
            o período solicitado e prepara os dados para uso posterior, como em modelos de aprendizado de máquina.
    '''
    
    def __is_ticker_valid__(self) -> bool:
        '''
            Description:
                Verifica se o ticker é considerado válido. Um ticker será considerado válido quando:
                - O atributo `data` não for vazio.
                - O índice inicial do atributo `data` corresponder à data de extração inicial (`data_extraction_initial_date`).
                - O índice final do atributo `data` corresponder à data de extração final (`data_extraction_final_date` - 1 dia).
            
            Args:
                Nenhum argumento é passado diretamente para esta função, pois ela usa os atributos do objeto 
                (`self.data`, `self.data_extraction_initial_date`, etc.).

            Return:
                bool: Retorna `True` se o ticker for considerado válido de acordo com as condições definidas, caso contrário retorna `False`.
        '''
        
        # Verifica se o atributo data é vazio.
        if(len(self.data) == 0): return False
        
        # Verifica se a data inicial do índice do DataFrame corresponde à data de extração inicial.
        condition_2 = (self.data.index[0].date() == self.data_extraction_initial_date)
        # Verifica se a data final do índice do DataFrame corresponde à data de extração final - 1 dia.
        condition_3 = (self.data.index[len(self.data.index)-1].date() == self.data_extraction_final_date - timedelta(days=1))
        
        # Observação: Irá ocorrer problemas com as condições acima caso self.data_extraction_initial_date ou self.data_extraction_final_data 
        # não sejam dias de negociação. Visto que, caso isso ocorra, tais condições serão falsas e consequentemente o Ticker em questão 
        # não será considerado válido.
        
        # Retorna False se alguma das condições não for satisfeita.
        if(not(condition_2) or not(condition_3)):
            return False
        
        # Retorna True caso contrário (se todas as condições forem satisfeitas).
        return True
        
        
    def __get_ticker_data__(self) -> pd.DataFrame:
        '''
            Description:
                Baixa os dados do ticker usando a biblioteca `yfinance` para o período de tempo definido pelos atributos 
                `data_extraction_initial_date` e `data_extraction_final_date`.
                
            Args:
                Nenhum argumento é passado diretamente para esta função, pois ela utiliza os atributos do objeto 
                (`self.symbol`, `self.data_extraction_initial_date`, etc.).
            
            Return:
                pd.DataFrame: Um DataFrame contendo os dados históricos do ticker para o período solicitado. 
                Retorna um DataFrame vazio se ocorrer algum erro.
        '''
        
        try:
            # Faz o download dos dados do ticker em questão.
            return yf.download(self.symbol, 
                               start=self.data_extraction_initial_date, 
                               end=self.data_extraction_final_date)
        except Exception as e:  
            # Captura e imprime a exceção caso ocorra um erro ao baixar os dados.
            print(f"Erro ao baixar os dados para {self.symbol}: {e}")
            # Retorna um DataFrame vazio em caso de erro.
            return pd.DataFrame()  
        
    def __get_arithmetic_returns__(self) -> pd.Series:
        '''
            Description:
                Calcula os retornos aritméticos baseados nos preços ajustados de fechamento (Adj Close).
                O retorno aritmético mede a variação percentual do preço de fechamento ajustado em relação ao período anterior.
                
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                pd.Series: Uma série com os retornos aritméticos para o período definido, divididos por 100 para normalizar os valores.
        '''
        
        # Obtém os preços ajustados de fechamento.
        daily_prices = self.data['Adj Close']

        # Calcula os retornos aritméticos usando o método ROC da talib.
        arithmetic_returns = talib.ROC(daily_prices, timeperiod=self.__features_time_period__['returns_time_period']) 
        
        # Normaliza os retornos dividindo por 100 (ROC retorna valores em percentual).
        return (arithmetic_returns/100)
    
    def __get_logarithmic_returns__(self) -> pd.Series:
        '''
            Description:
                Calcula os retornos logarítmicos baseados nos retornos aritméticos. 
                O retorno logarítmico é uma medida que transforma os retornos aritméticos em uma forma que facilita a análise estatística e 
                agregação de múltiplos retornos.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                pd.Series: Uma série com os retornos logarítmicos.
        '''
        
        # Obtém os retornos aritméticos.
        arithmetic_returns = self.__get_arithmetic_returns__()
        
        # Calcula os retornos logarítmicos aplicando a função log(1 + retorno aritmético).
        logarithmic_returns = arithmetic_returns.apply(lambda x: np.log(1 + x))
        
        return logarithmic_returns
    
    def __get_bollinger_bands_with_exponential_moving_average__(self) -> pd.Series:
        '''
            Description:
                Calcula as Bandas de Bollinger com base na média móvel exponencial dos preços ajustados de fechamento (Adj Close).
                As Bandas de Bollinger são usadas para medir a volatilidade e identificar potenciais pontos de reversão de tendência.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                Tuple[pd.Series, pd.Series, pd.Series]: Um tuple contendo três séries: a banda superior, a banda média (EMA) e a banda inferior.
        '''
        
        # Obtém os preços ajustados de fechamento.
        daily_prices = self.data['Adj Close']
        
        # Calcula as Bandas de Bollinger usando o talib.
        upper_band, middle_band, lower_band = talib.BBANDS(daily_prices, 
                                                           timeperiod=self.__features_time_period__['exponential_moving_average_time_period'],
                                                           nbdevup=2, nbdevdn=2, matype=1) # Tipo de média móvel: 1 é a média móvel exponencial.
        
        return upper_band, middle_band, lower_band
        
    def __get_relative_strength_index__(self) -> pd.Series:
        '''
            Description:
                Calcula o Índice de Força Relativa (RSI) com base nos preços ajustados de fechamento (Adj Close).
                O RSI é um indicador de momentum usado para identificar condições de sobrecompra ou sobrevenda de um ativo.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                pd.Series: Uma série contendo os valores do RSI para o período definido.
        '''
        
        # Obtém os preços ajustados de fechamento.
        daily_prices = self.data['Adj Close']
        
        # Calcula o RSI usando o talib.
        relative_strength_index = talib.RSI(daily_prices, timeperiod=self.__features_time_period__['relative_strength_index_time_period'])
        
        return relative_strength_index
    
    def __get_average_true_range__(self) -> pd.Series:
        '''
            Description:
                Calcula o Average True Range (ATR) baseado nas séries de preços máximos, mínimos e de fechamento.
                O ATR é um indicador de volatilidade que mede a variação média entre o preço máximo e o preço mínimo do período.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                pd.Series: Uma série contendo os valores do ATR para o período definido.
        '''
        
        # Obtém as séries de preços máximos, mínimos e de fechamento.
        high_prices = self.data['High']
        low_prices = self.data['Low']
        close_prices = self.data['Close']
        
        # Calcula o ATR usando o talib.
        average_true_range = talib.ATR(high_prices, low_prices, close_prices, timeperiod=self.__features_time_period__['average_true_range_time_period'])

        return average_true_range
    
    def __get_momemtum__(self) -> pd.Series:
        '''
            Description:
                Calcula o indicador de momentum baseado nos preços ajustados de fechamento (Adj Close).
                O momentum mede a taxa de variação dos preços, indicando a velocidade e direção das mudanças de preços.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                pd.Series: Uma série contendo os valores de momentum para o período definido.
        '''
        
        # Obtém os preços ajustados de fechamento.
        daily_prices = self.data['Adj Close']
        
        # Calcula o momentum usando o talib
        momemtum = talib.MOM(daily_prices, timeperiod=self.__features_time_period__['momemtum_time_period'])

        return momemtum
        
    
    def __init__(self, symbol: str, data_extraction_initial_date: datetime.date , data_extraction_final_date: datetime.date,
                 features_time_period: dict) -> None:
        '''
            Description:
                Inicializa uma instância da classe "Ticker", definindo os parâmetros essenciais como o símbolo do ativo financeiro,
                o período de extração de dados e os períodos de tempo utilizados para calcular indicadores financeiros. Durante a inicialização,
                os dados do ticker são baixados e validados.

            Args:
                symbol (str): O símbolo do ativo financeiro (ticker) cujos dados históricos serão extraídos.
                data_extraction_initial_date (datetime.date): A data inicial para a extração dos dados.
                data_extraction_final_date (datetime.date): A data final para a extração dos dados.
                features_time_period (dict): Um dicionário que contém os parâmetros de tempo para o cálculo dos indicadores financeiros, 
                                            como RSI, Bandas de Bollinger, etc.
            
            Return:
                None: O construtor não retorna nada, mas prepara a instância para que os dados e indicadores possam ser utilizados posteriormente.
        '''
        
        # Define o símbolo do ticker.
        self.symbol = symbol
        
        # Define a data inicial para extração dos dados.
        self.data_extraction_initial_date = data_extraction_initial_date
        
        # Define a data final para extração dos dados.
        self.data_extraction_final_date = data_extraction_final_date
        
        # Define os períodos de tempo para calcular os indicadores financeiros.
        self.__features_time_period__ = features_time_period
        
        # Baixa os dados do ticker.
        self.data = self.__get_ticker_data__()
        
        # Verifica se o ticker é válido.
        self.is_valid = self.__is_ticker_valid__()
    
    def __set_features__(self) -> None:
        '''
            Description:
                Calcula e adiciona alguns indicadores financeiros como colunas ao DataFrame 'data' da instância. 
                Esses indicadores incluem retornos logarítmicos, Bandas de Bollinger com média móvel exponencial, RSI, ATR e Momentum.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                None: A função não retorna nada, mas modifica o DataFrame 'data' da instância.
        '''
        
        # Adiciona os retornos logarítmicos ao DataFrame.
        self.data['Log returns'] = self.__get_logarithmic_returns__()
        
        # Adiciona as Bandas de Bollinger (banda superior, EMA, banda inferior) ao DataFrame.
        self.data['B. upper bands'], self.data['EMA'], self.data['B. lower bands'] = self.__get_bollinger_bands_with_exponential_moving_average__()
        
        # Adiciona o Índice de Força Relativa (RSI) ao DataFrame.
        self.data['RSI'] = self.__get_relative_strength_index__()
        
        # Adiciona o Average True Range (ATR) ao DataFrame.
        self.data['ATR'] = self.__get_average_true_range__()
        
        # Adiciona o indicador de Momentum ao DataFrame.
        self.data['MOM'] = self.__get_momemtum__()
    
    def __remove_some_features__(self) -> None:
        '''
            Description:
                Remove algumas colunas desnecessárias do DataFrame, especificamente as colunas 'Open', 'High', 'Low', e 'Close'.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                None: A função não retorna nada, mas modifica o DataFrame 'data' da instância.
        '''
        
        # Remove as colunas 'Open', 'High', 'Low' e 'Close' do DataFrame.
        self.data.drop(columns=['Open','High','Low','Close'], inplace=True)
        
    def adjust_data(self) -> None:
        '''
            Description:
                Prepara os dados do ticker, calculando e adicionando os indicadores financeiros (features) e realizando
                a limpeza dos dados, caso o ticker seja válido. Este método é útil para garantir que o DataFrame 'data'
                contenha as informações necessárias para análises subsequentes.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                None: A função não retorna nada, mas modifica o DataFrame 'data' da instância ao adicionar indicadores e
                remover colunas e linhas desnecessárias.
        '''
        
        # Se o ticker for considerado válido:
        if self.is_valid:
            # Calcula e adiciona as features (indicadores financeiros) ao DataFrame.
            self.__set_features__()
            
            # Remove algumas colunas que não serão necessárias.
            self.__remove_some_features__()
            
            # Remove todas as linhas que contenham valores nulos (NaN) no DataFrame.
            self.data.dropna(inplace=True)
            
    def __train_test_split__(self, test_initial_day: datetime.date, test_final_day: datetime.date) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        '''
            Description:
                Divide os dados em conjuntos de treino e teste com base nas datas fornecidas. As features (X) e o alvo (y) são separados
                e divididos em dados de treino e teste.

            Args:
                test_initial_day (datetime.date): Data que define o início do período de teste.
                test_final_day (datetime.date): Data que define o final do período de teste.

            Return:
                tuple: Retorna quatro elementos - X_train (features de treino), X_test (features de teste), y_train (alvo de treino) e y_test (alvo de teste).
        '''
        
        # Separa as features (X) e o target (y).
        X = self.data.drop(columns=['Adj Close'])
        y = self.data['Adj Close']
        
        # Separa o conjunto de séries temporais das features em dados de treino e dados de teste.
        X_train, X_test = X[X.index < test_initial_day], X[(X.index >= test_initial_day) & (X.index <= test_final_day)]
        
        # Separa a série temporal do target em dados de treino e dados de teste.
        y_train, y_test = y[y.index < test_initial_day], y[(y.index >= test_initial_day) & (y.index <= test_final_day)]

        return X_train, X_test, y_train, y_test
    
    def __scale_data__(self, X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
        '''
            Description:
                Normaliza as features e o target tanto para os conjuntos de treino quanto de teste usando o MinMaxScaler. Além disso,
                redimensiona o alvo (y) para uma matriz bidimensional antes da normalização.

            Args:
                X_train (pd.DataFrame): Conjunto de treino das features.
                X_test (pd.DataFrame): Conjunto de teste das features.
                y_train (pd.Series): Conjunto de treino do alvo.
                y_test (pd.Series): Conjunto de teste do alvo.

            Return:
                tuple: Retorna quatro elementos - X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled.
        '''
        
        # Cria uma instancia do MinMaxScaler para as features do ticker em questão.
        scaler_features = MinMaxScaler()

        # Ajusta a instância criada acima ao conjunto de treino das features do ticker em questão.
        X_train_scaled = scaler_features.fit_transform(X_train)  

        # Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
        resized_y_train = y_train.values.reshape(-1, 1)  

        # Cria uma instancia do MinMaxScaler para o target do ticker em questão.
        scaler_target = MinMaxScaler()

        # Ajusta a instância criada acima ao conjunto de treino do target do ticker em questão.
        y_train_scaled = scaler_target.fit_transform(resized_y_train)  

        # Normaliza o conjunto de teste das features do ticker em questão usando a instância que foi criada e ajustada aos dados de treino
        # desse mesmo ticker.
        X_test_scaled = scaler_features.transform(X_test)  

        # Redimensiona y_test para que seja bidimensional (Necessário para o MinMaxScaler).
        resized_y_test = y_test.values.reshape(-1, 1) 

        # Normaliza o conjunto de teste do target  do ticker em questão usando a instância que foi criada e ajustada aos dados de treino desse
        # mesmo ticker.
        y_test_scaled = scaler_target.transform(resized_y_test)
        
        return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled
    
    def __create_time_sequences_for_lstm__(self, X_train_scaled: np.ndarray, X_test_scaled: np.ndarray, y_train_scaled: np.ndarray,
                              y_test_scaled: np.ndarray, sequence_length: int) -> tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
        '''
            Description:
                Constrói sequências temporais a partir dos dados normalizados de treino e teste, criando janelas móveis
                de tamanho 'sequence_length'. Essas sequências são necessárias para treinar uma LSTM.

            Args:
                X_train_scaled (np.ndarray): Conjunto de treino normalizado das features.
                X_test_scaled (np.ndarray): Conjunto de teste normalizado das features.
                y_train_scaled (np.ndarray): Conjunto de treino normalizado do alvo.
                y_test_scaled (np.ndarray): Conjunto de teste normalizado do alvo.
                sequence_length (int): O comprimento das sequências temporais usadas para treinar a LSTM.

            Return:
                tuple: Retorna quatro elementos - X_train_scaled_sequences, X_test_scaled_sequences, 
                    y_train_scaled_sequences, y_test_scaled_sequences (sequências temporais normalizadas para treino e teste).
        '''
    
        # Verifica se o sequence_length é maior que o número de amostras disponíveis
        if sequence_length > len(X_train_scaled):
            raise ValueError("O 'sequence_length' não pode ser maior que o número de amostras em 'X_train_scaled'.")
        if sequence_length > len(X_test_scaled):
            raise ValueError("O 'sequence_length' não pode ser maior que o número de amostras em 'X_test_scaled'.")
        
        # Calcula o comprimento dos intervalos de treino e teste, assumindo que 'X_train' e 'y_train' têm o mesmo comprimento, tal como 'X_test' e
        # 'y_test'.
        train_interval_length = len(X_train_scaled) - sequence_length
        test_interval_length = len(X_test_scaled) - sequence_length
        
        # Inicializa listas para armazenar as sequências temporais dos dados de treino e teste.
        X_train_sequences = []
        X_test_sequences = []
        y_train_sequences = []
        y_test_sequences = []
    
        # Cria sequências temporais para os dados de treino
        for i in range(train_interval_length):
            # Cria uma sequência temporal de 'sequence_length' dias para as features de treino.
            X_train_sequence = X_train_scaled[i: (i + sequence_length)]
            # O alvo será o valor no dia seguinte após a sequência temporal.
            y_train_sequence = y_train_scaled[i + sequence_length]
            # Adiciona as sequências temporais às listas correspondentes.
            X_train_sequences.append(X_train_sequence)
            y_train_sequences.append(y_train_sequence)

        # Cria sequências temporais para os dados de teste
        for j in range(test_interval_length):
            # Cria uma sequência temporal de 'sequence_length' dias para as features de teste.
            X_test_sequence = X_test_scaled[j: (j + sequence_length)]
            # O alvo será o valor no dia seguinte após a sequência temporal.
            y_test_sequence = y_test_scaled[j + sequence_length]
            # Adiciona as sequências temporais às listas correspondentes.
            X_test_sequences.append(X_test_sequence)
            y_test_sequences.append(y_test_sequence)
            
        # Converte as listas em arrays.
        X_train_scaled_sequences = np.array([np.array(arr) for arr in X_train_sequences])
        X_test_scaled_sequences = np.array([np.array(arr) for arr in X_test_sequences])
        y_train_scaled_sequences = np.array(y_train_sequences)
        y_test_scaled_sequences = np.array(y_test_sequences)
        
        return X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences
    
    def prepare_data_for_lstm(self, test_initial_day: datetime.date, test_final_day: datetime.date, lstm_time_sequences_length: int) -> tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
        '''
            Description:
                Prepara os dados para serem usados em uma LSTM. O processo envolve dividir os dados em conjuntos de treino e teste,
                normalizar os dados e criar sequências temporais para a LSTM.
                
                Observação: A rede neural será treinada com todos os dados antes da data "test_initial_day" e fará predições para 
                            todas as datas entre test_initial_day e test_final_day (inclusos).

            Args:
                test_initial_day (datetime.date): Data que define o início do período de teste.
                test_final_day (datetime.date): Data que define o final do período de teste.
                lstm_time_sequences_length (int): O comprimento das sequências temporais usadas para treinar a LSTM.

            Return:
                tuple: Retorna quatro elementos - X_train_scaled_sequences, X_test_scaled_sequences, 
                    y_train_scaled_sequences, y_test_scaled_sequences (sequências temporais normalizadas para treino e teste).
        '''

        
        # Divide os dados em conjuntos de treino e teste.
        X_train, X_test, y_train, y_test = self.__train_test_split__(test_initial_day, test_final_day)
        
        # Normaliza os dados.
        X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = self.__scale_data__(X_train, X_test, y_train, y_test)
        
        # Cria sequências temporais para a LSTM.
        X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences = self.__create_time_sequences_for_lstm__(X_train_scaled, X_test_scaled,
                                                                                                                   y_train_scaled, y_test_scaled,
                                                                                                                   lstm_time_sequences_length)
        
        return X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences

# **Definindo a classe Tickers**

In [348]:
class Tickers:
    '''
        Description:
            A classe "Tickers" serve como um contêiner para armazenar e padronizar múltiplos objetos "Ticker", garantindo que todos os
            dados extraídos dos tickers tenham o mesmo intervalo de datas de negociação. Isso é útil para análises financeiras que
            exigem comparabilidade entre diferentes ativos ao longo de um período comum. A classe também valida os tickers e armazena
            apenas aqueles que possuem dados válidos.
    '''
    
    def __get_tickers_data__(self) -> None:
        '''
            Description:
                Itera sobre a lista de símbolos de tickers (tickers_list), cria um objeto Ticker para cada símbolo e salva os dados 
                no atributo "data" caso o ticker seja válido.
            
            Args:
                Nenhum argumento é passado diretamente, pois a função utiliza os atributos do objeto.
            
            Return:
                None: A função não retorna nada, mas preenche o array "data" com os objetos "Ticker" válidos.
        '''
        
        # Itera sobre cada um dos símbolos presentes em "tickers_list".
        for i, symbol in enumerate(self.symbols_list):
            # Cria um objeto "Ticker" para o símbolo atual.
            ticker = Ticker(symbol, self.data_extraction_initial_date, self.data_extraction_final_date, setup['features_time_period'])
            # Se o ticker for válido, salva o objeto "Ticker" no array "data".
            if(ticker.is_valid): self.symbols[i] = ticker
    
    def __init__(self, symbols_list: list, data_extraction_initial_date: datetime.date,
                 data_extraction_final_date: datetime.date) -> None:
        '''
            Description:
                Inicializa a classe "Tickers", criando uma lista de objetos "Ticker" com base na lista de símbolos e nas datas de extração
                fornecidas. A classe também armazena os objetos "Ticker" válidos no atributo "data".
            
            Args:
                symbols_list (list): A lista de símbolos (tickers) para os quais os dados serão extraídos.
                data_extraction_initial_date (datetime.date): A data inicial para a extração dos dados.
                data_extraction_final_date (datetime.date): A data final para a extração dos dados.
            
            Return:
                None: A função não retorna nada, mas inicializa a instância com os tickers válidos e seus dados.
        '''
        
        # Cria um array que guardará os objetos "Ticker" válidos.
        self.symbols =  np.empty(len(symbols_list), dtype=object)

        # Define a lista de símbolos de tickers.
        self.symbols_list = symbols_list
        
        # Define a data inicial para a extração dos dados.
        self.data_extraction_initial_date = data_extraction_initial_date
        
        # Define a data final para a extração dos dados.
        self.data_extraction_final_date = data_extraction_final_date
        
        # Preenche o array "tickers" com os tickers válidos.
        self.__get_tickers_data__()
        
        # Remove os valores nulos (None) do array "data".
        self.symbols = self.symbols[self.tickers != None]

# **Definindo os parâmetros iniciais**

#### *Obtendo a lista de tickers do S&P500*

In [349]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_symbols= sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_symbols = sp500_symbols.tolist()

# Exibe a lista de tickets criada acima.
sp500_symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMTM',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',

#### *Definindo alguns parâmetros que serão importantes*

In [397]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais que este código necessita. 
'''

# Cria um dicionário para guardar alguns parâmetros importantes.
setup = {
    #
    "symbols": sp500_symbols,
    # Define a data inicial cujos dados serão coletados (Primeiro dia de negociações do S&P500 em 2019).
    "data_extraction_initial_date": datetime(2019,1,2).date(), # Deve obrigatoriamente ser um dia de negociação.
    # Define a data final cujos dados serão coletados (Último dia de negociações do S&P500 em 2023).
    "data_extraction_final_date": datetime(2023,12,30).date(), # Deve obrigatoriamente ser um dia de negociação e deve também obrigatoriamente
                                                               # suceder um dia de negociação.
    "features_time_period": {
        #
        "returns_time_period": 1,
        #
        "exponential_moving_average_time_period": 14,
        #
        "relative_strength_index_time_period": 14,
        #
        "average_true_range_time_period": 14,
        #
        "momemtum_time_period": 14
    },
    "lstm_sequences_length": 14
}

# **Criando a rede neural**

In [403]:
"""
    Este código define, constrói e compila um modelo LSTM (Long Short-Term Memory) para previsões de séries temporais.

    Parâmetros dos métodos utilizados:
    1. `LSTM`:
        - `units`: Número de neurônios na camada LSTM.
        - `return_sequences`: Se `True`, retorna a sequência completa de saídas para cada unidade LSTM; se `False`, retorna apenas 
           a saída final.
        - `input_shape`: Tupla que define a forma da entrada (tamanho da sequência de tempo, número de features).

    2. `Dropout`:
        - `rate`: Fração de neurônios a serem descartados durante o treinamento para evitar overfitting.

    3. `Dense`:
        - `units`: Número de neurônios na camada densa (totalmente conectada). No contexto de regressão, geralmente é 1.

    4. `compile`:
        - `optimizer`: Algoritmo de otimização utilizado para ajustar os pesos do modelo. 'adam' é uma escolha comum para LSTM.
        - `loss`: Função de perda que o modelo tentará minimizar durante o treinamento. Neste caso, 'mean_squared_error' (erro quadrático médio) é usado para problemas de regressão.
"""

def create_LSTM_model(features_number: int, lstm_sequence_length: int):
    '''
        Description:

        Args:

        Return:

    '''

    # Definindo o modelo LSTM
    # Utilizamos o modelo 'Sequential', que permite empilhar camadas de forma linear. Esse tipo de modelo é adequado para a maioria das 
    # arquiteturas de rede neural onde as camadas são adicionadas uma após a outra.
    model = Sequential()

    # Adiciona a camada de entrada explicitamente
    model.add(Input(shape=(lstm_sequence_length, features_number)))

    # Adiciona a primeira camada LSTM com 64 unidades (neurônios).
    model.add(LSTM(units=64, return_sequences=True))

    # Adiciona uma camada de Dropout. Tal camada é usada para evitar overfitting durante o treinamento do modelo. Ao definir `rate=0.2`, 
    # estamos especificando que 20% dos neurônios da camada anterior serão desligados aleatoriamente em cada atualização do ciclo de treinamento.
    model.add(Dropout(0.2))

    # Adiciona uma segunda LSTM. Ao definirmos`return_sequences=False` indicamos que esta é a última camada LSTM na rede, e 
    # ela só retorna a última saída em vez de toda a sequência.
    model.add(LSTM(units=128, return_sequences=False))

    # Adiciona uma segunda camada de Dropout. Assim como antes, 20% dos neurônios serão desativados em cada iteração de treinamento.
    model.add(Dropout(0.2))

    # Adiciona a camada de saída. Tal camada é uma camada densa totalmente conectada com um único neurônio (units=1). Isso é adequado para 
    # problemas de regressão onde a previsão final é um único valor contínuo.
    model.add(Dense(units=1))

    # Compila o modelo criado. O modelo é compilado com o otimizador 'adam', que é eficiente para grandes volumes de dados e adequado para
    # problemas de regressão. Além disso, setamos a função de perda como sendo a 'mean_squared_error' (MSE), que é uma escolha comum para medir 
    # o erro médio ao quadrado entre as previsões e os valores reais.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # A variável 'model' agora contém o modelo LSTM compilado e pode ser usada para treinamento e previsões. Logo, retornamos ela
    return model

In [412]:
def train_model_and_get_results(model: Sequential, X_train_scaled_sequences, X_test_scaled_sequences ,
                                y_train_scaled_sequences, y_test_scaled_sequences) -> tuple[np.ndarray, float]:
   '''
      Description:
      
      Args:

      Return:
   '''
      
   """
      Este código treina o modelo LSTM usando o método `fit` do módulo Keras da biblioteca tensorflow.

      Parâmetros do método `fit`:
      1. `x`: Dados de entrada para treinamento. No contexto de séries temporais com LSTM, este é um array de sequências escaladas.
      2. `y`: Dados de saída (alvo) para treinamento. Correspondem aos valores que o modelo deve prever com base nos dados de entrada.
      3. `epochs`: Número de vezes que o modelo passará por todo o conjunto de dados de treinamento. Um número maior de épocas pode melhorar o 
         ajuste do modelo, mas também aumenta o risco de overfitting.
      4. `batch_size`: Número de amostras que o modelo processa antes de atualizar os pesos. Tamanhos de batch menores podem resultar em um 
         treinamento mais ruidoso, mas permitem uma melhor generalização.
      5. `validation_split`: Proporção dos dados de treinamento que será usada para validação. Ajuda a monitorar a performance do modelo em 
         dados que ele não viu durante o treinamento.
      6. `verbose`: Nível de verbosidade do processo de treinamento. `verbose=1` exibe uma barra de progresso detalhada durante o treinamento.
   """

   # Treina o modelo
   # O método `fit` treina o modelo LSTM usando os dados de entrada (`X_train_scaled_sequence`) e as saídas correspondentes 
   # (`y_train_scaled_sequence`).
   model.fit(
      X_train_scaled_sequences,  # Dados de entrada de treino (sequências temporais)
      y_train_scaled_sequences,  # Dados de saída de treino (valores de previsão para cada sequência)
      epochs=50,  # Número de épocas: o modelo passará 50 vezes por todo o conjunto de dados de treinamento
      batch_size=32,  # Tamanho do batch: o modelo ajusta os pesos após processar cada lote de 32 amostras
      validation_split=0.2,  # 20% dos dados de treinamento serão usados para validação.
      verbose=1  # Nível de verbosidade: 1 mostra uma barra de progresso e resultados após cada época
   )
   
   """
         Este bloco de código faz previsões sobre um conjunto de dados de teste usando o modelo LSTM treinado acima, e, em seguida,
         avalia a precisão dessas previsões usando a métrica de Root Mean Squared Error (RMSE).
   """

   
   # Realiza a previsão de valores para dados não vistos até então.
   predicted = model.predict(X_test_scaled_sequences)

   # Avalia, utilizando a métrica RMSE, o resultado das previsões feitas acima.
   RMSE = np.sqrt(mean_squared_error(y_test_scaled_sequences,predicted))

   return predicted, RMSE

# **Testes**

In [367]:
ticker = Ticker('MMM', setup['data_extraction_initial_date'], setup['data_extraction_final_date'], setup['features_time_period'])

[*********************100%%**********************]  1 of 1 completed


In [368]:
# Exibe os dados do ticker de teste antes deles serem "ajustados".
ticker.data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,157.040131,159.690628,156.103683,159.657196,126.860451,2960339
2019-01-03,157.424744,157.424744,152.918060,153.645493,122.083618,4016407
2019-01-04,156.145493,160.518402,155.543472,159.966553,127.106232,3582140
2019-01-07,160.000000,160.785950,157.742477,159.598663,126.813919,2585991
2019-01-08,161.371231,162.299332,158.511703,160.267563,127.345436,2965841
...,...,...,...,...,...,...
2023-12-22,88.528427,89.807693,88.419731,88.904678,86.383255,2728913
2023-12-26,88.879601,90.794312,88.854515,90.392975,87.829346,3332176
2023-12-27,90.301003,91.220734,90.033447,90.919731,88.341164,2922785


In [369]:
# Realiza o "ajuste" dos dados para o ticker em questão.
ticker.adjust_data()

In [375]:
# Exibe os dados do ticker em questão após os mesmos terem sido "ajustados".
ticker.data.head(516)

,Adj Close,Volume,Log returns,B. upper bands,EMA,B. lower bands,RSI,ATR,MOM
Date,,,,,,,,,
2019-01-23,127.750694,1976868,0.000156,130.629937,127.136236,123.642535,52.022916,3.724319,0.890244
2019-01-24,127.903442,1893029,0.001195,129.357918,127.238530,125.119142,52.378923,3.557437,5.819824
2019-01-25,130.148987,2774840,0.017404,130.121977,127.626591,125.131205,57.385210,3.580450,3.042755
2019-01-28,128.355225,3775294,-0.013878,130.182002,127.723742,125.265482,52.625900,3.613165,1.541306
2019-01-29,130.846603,5156076,0.019224,131.019168,128.140124,125.261080,57.854172,3.754628,3.501167
...,...,...,...,...,...,...,...,...,...
2021-02-02,124.670410,3401902,0.001830,132.829676,124.215668,115.601661,52.884711,3.415187,6.061829
2021-02-03,124.998085,3437543,0.002625,132.579895,124.319991,116.060087,53.542601,3.403566,6.553329
2021-02-04,126.152054,2192268,0.009190,132.425150,124.564266,116.703382,55.879131,3.268552,7.614700


In [378]:
test_initial_day = pd.to_datetime(datetime(2021,1,19))
test_final_day = pd.to_datetime(datetime(2021,2,8))

X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences = ticker.prepare_data_for_lstm(test_initial_day, test_final_day, 14)

In [396]:
features_number = len(X_train_scaled_sequences[0][0])

In [405]:
model = create_LSTM_model(features_number, setup['lstm_sequences_length'])

In [406]:
predicted, RMSE = train_model_and_get_results(model, X_train_scaled_sequences, X_test_scaled_sequences, y_train_scaled_sequences, y_test_scaled_sequences)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - loss: 0.0644 - val_loss: 0.0043
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0111 - val_loss: 0.0056
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0089 - val_loss: 0.0024
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0067 - val_loss: 0.0022
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0066 - val_loss: 0.0028
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0069 - val_loss: 0.0021
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0062 - val_loss: 0.0022
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0049 - val_loss: 0.0021
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0067 - val_loss: 0.0032
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0058 - val_loss: 0.0023
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0042 - val_loss: 0.0015
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.

In [409]:
type(predicted)

numpy.ndarray

In [410]:
RMSE

0.04736078249112663